In [1]:
import json
from collections import defaultdict()

In [4]:
def rename_metric(metric):
    if metric == 'F-measure':
        return 'f1'
    if metric == 'Accuracy':
        return 'acc'
    if metric == 'Runtime':
        return 'time'
    raise ValueError('Unknown metric')

In [58]:
def get_results(metric):
    Res = {metric: {'Carcinogenesis': {'CELOE': '', 'ELTL': '', 'ECII': '', 'EvoLearner': '', '\\textbf{$\\textbf{NCES}_{LSTM}$}': '', '\\textbf{$\\textbf{NCES}_{GRU}$}': '',\
                                       '\\textbf{$\\textbf{NCES}_{ST}$}': '', '\\textbf{$\\textbf{NCES}_{ST+GRU}$}': '', '\\textbf{$\\textbf{NCES}_{ST+LSTM}$}': '',\
                                       '\\textbf{$\\textbf{NCES}_{GRU+LSTM}$}': '', '\\textbf{$\\textbf{NCES}_{ST+GRU+LSTM}$}': ''},
                   'Mutagenesis': {'CELOE': '', 'ELTL': '', 'ECII': '', 'EvoLearner': '', '\\textbf{$\\textbf{NCES}_{LSTM}$}': '', '\\textbf{$\\textbf{NCES}_{GRU}$}': '',\
                                       '\\textbf{$\\textbf{NCES}_{ST}$}': '', '\\textbf{$\\textbf{NCES}_{ST+GRU}$}': '', '\\textbf{$\\textbf{NCES}_{ST+LSTM}$}': '',\
                                       '\\textbf{$\\textbf{NCES}_{GRU+LSTM}$}': '', '\\textbf{$\\textbf{NCES}_{ST+GRU+LSTM}$}': ''},
                    'Semantic Bible': {'CELOE': '', 'ELTL': '', 'ECII': '', 'EvoLearner': '', '\\textbf{$\\textbf{NCES}_{LSTM}$}': '', '\\textbf{$\\textbf{NCES}_{GRU}$}': '',\
                                       '\\textbf{$\\textbf{NCES}_{ST}$}': '', '\\textbf{$\\textbf{NCES}_{ST+GRU}$}': '', '\\textbf{$\\textbf{NCES}_{ST+LSTM}$}': '',\
                                       '\\textbf{$\\textbf{NCES}_{GRU+LSTM}$}': '', '\\textbf{$\\textbf{NCES}_{ST+GRU+LSTM}$}': ''},
                    'Vicodi': {'CELOE': '', 'ELTL': '', 'ECII': '', 'EvoLearner': '', '\\textbf{$\\textbf{NCES}_{LSTM}$}': '', '\\textbf{$\\textbf{NCES}_{GRU}$}': '',\
                                       '\\textbf{$\\textbf{NCES}_{ST}$}': '', '\\textbf{$\\textbf{NCES}_{ST+GRU}$}': '', '\\textbf{$\\textbf{NCES}_{ST+LSTM}$}': '',\
                                       '\\textbf{$\\textbf{NCES}_{GRU+LSTM}$}': '', '\\textbf{$\\textbf{NCES}_{ST+GRU+LSTM}$}': ''}
                   }}
    for kb in ['carcinogenesis', 'mutagenesis', 'semantic_bible', 'vicodi']:
        for model in ['celoe', 'eltl', 'ecii', 'evolearner', 'NCES', 'NCES_Ensemble']:
            if model not in ['NCES', 'NCES_Ensemble']:
                with open(f'datasets/{kb}/Results/concept_learning_avg_results__{model}.json') as file:
                    results = json.load(file)
                rename_model = model.upper() if model != 'evolearner' else 'EvoLearner'
                if model == 'ecii':
                    if metric != 'Accuracy':
                        Res[metric][' '.join(word.capitalize() for word in kb.split('_'))][rename_model] =\
                    str(round(100*results[metric]['mean'],2))+'$\pm$'+str(round(100*results[metric]['std'],2))
                    else:
                        Res[metric][' '.join(word.capitalize() for word in kb.split('_'))][rename_model] =\
                    '-'+'$\pm$'+'-'
                else:
                    Res[metric][' '.join(word.capitalize() for word in kb.split('_'))][rename_model] =\
                    str(round(results[metric]['mean'],2))+'$\pm$'+str(round(results[metric]['std'],2))
            elif model == 'NCES':
                with open(f'datasets/{kb}/Results/{model}.json') as file:
                    results = json.load(file)
                new_metric = rename_metric(metric)
                for m in results:
                    rename_model = '\\textbf{$\\textbf{NCES}_'+'{'+m+'}$}' if m != 'SetTransformer' else '\\textbf{$\\textbf{NCES}_'+'{'+'ST'+'}$}'
                    Res[metric][' '.join(word.capitalize() for word in kb.split('_'))][rename_model] =\
                    str(round(results[m][new_metric]['mean'][0],2))+'$\pm$'+str(round(results[m][new_metric]['std'][0],2))
            elif model == 'NCES_Ensemble':
                with open(f'datasets/{kb}/Results/{model}.json') as file:
                    results = json.load(file)
                new_metric = rename_metric(metric)
                for m in results:
                    rename_model = '\\textbf{$\\textbf{NCES}_'+'{'+m+'}$}' if not 'SetTransformer' in m \
                    else '\\textbf{$\\textbf{NCES}_'+'{'+m.replace('SetTransformer', 'ST')+'}$}'
                    Res[metric][' '.join(word.capitalize() for word in kb.split('_'))][rename_model] =\
                    str(round(results[m][new_metric]['mean'][0],2))+'$\pm$'+str(round(results[m][new_metric]['std'][0],2))
    return Res

In [59]:
Res1 = get_results('F-measure')

In [60]:
Res1

{'F-measure': {'Carcinogenesis': {'CELOE': '59.06$\\pm$38.61',
   'ELTL': '40.36$\\pm$31.94',
   'ECII': '33.1$\\pm$37.12',
   'EvoLearner': '85.19$\\pm$21.51',
   '\\textbf{$\\textbf{NCES}_{LSTM}$}': '82.46$\\pm$25.41',
   '\\textbf{$\\textbf{NCES}_{GRU}$}': '86.53$\\pm$26.27',
   '\\textbf{$\\textbf{NCES}_{ST}$}': '85.44$\\pm$27.92',
   '\\textbf{$\\textbf{NCES}_{ST+GRU}$}': '97.11$\\pm$9.6',
   '\\textbf{$\\textbf{NCES}_{ST+LSTM}$}': '93.94$\\pm$16.34',
   '\\textbf{$\\textbf{NCES}_{GRU+LSTM}$}': '87.74$\\pm$24.07',
   '\\textbf{$\\textbf{NCES}_{ST+GRU+LSTM}$}': '96.13$\\pm$10.65'},
  'Mutagenesis': {'CELOE': '96.43$\\pm$15.81',
   'ELTL': '33.06$\\pm$31.22',
   'ECII': '26.37$\\pm$35.45',
   'EvoLearner': '95.18$\\pm$6.19',
   '\\textbf{$\\textbf{NCES}_{LSTM}$}': '67.11$\\pm$45.87',
   '\\textbf{$\\textbf{NCES}_{GRU}$}': '64.21$\\pm$44.75',
   '\\textbf{$\\textbf{NCES}_{ST}$}': '88.97$\\pm$25.35',
   '\\textbf{$\\textbf{NCES}_{ST+GRU}$}': '90.58$\\pm$26.0',
   '\\textbf{$\\textbf{N

In [63]:
with open(f'F1.txt', 'w') as file:
    file.write("\\begin{tabular}{@{}lccccccc@{}}\n")
    file.write("(\%)$}\\\\")

In [73]:
def write_results(metric):
    data = get_results(metric)
    with open(f'{metric}.txt', 'w') as file:
        for key in data:
            file.write("\\begin{tabular}{@{}lccccccc@{}}\n")
            file.write("\t\t\\toprule\n")
            file.write("\t\t& \\multicolumn{4}{c}{"+"$"+metric+"(\%)$}\\\\\n")
            file.write("\t\t&"+" & ".join(["\\textbf{"+kb+"}" for kb in data[key]])+"\\\\\n")
            file.write("\\midrule\n")
            for model in data[key]['Carcinogenesis']:
                file.write("\t\t"+model+" & "+" & ".join([data[key][kb][model] for kb in data[key]])+"\\\\\n")

In [74]:
write_results('F-measure')

In [75]:
write_results('Accuracy')

In [76]:
write_results('Runtime')